In [369]:
from flask import Flask, request, jsonify, render_template
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [370]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [371]:
from Income import *
from app import *

In [ ]:
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/run_algorithm', methods=['POST'])
def run_algorithm():
    try:
        data = request.get_json()
        print('Received data:', data)
        result = process_data(data)
        print('Result:', result)
        return jsonify({'result': result})
    except Exception as e:
        return jsonify({'error': str(e)})
    
def process_data(data):
    # From Income file
    ticker = data['input']
    URL = f'https://www.marketwatch.com/investing/stock/{ticker}/financials/income/quarter'
    html = requests.get(URL)
    soup = BeautifulSoup(html.content, "html.parser")
    table = soup.find('table', class_ = 'table table--overflow align--right').text.split('\n')
    table = [element for element in table if element != '' and element != ' ']
    table.remove('5- qtr trend')

    dic = {}
    for i in range(0, len(table), 7):
        dic[table[i]] = table[i+2: i+7]

    quarter_df = pd.DataFrame(dic).replace('-', np.NaN)
    quarter_df.rename(columns = {'Item': 'Period Ending'}, inplace=True)
    quarter_df = quarter_df.set_index('Period Ending')

    for col in quarter_df.columns:
        quarter_df[col] = quarter_df[col].apply(convert_to_float)

    quarter_df = quarter_df.reset_index()
    return quarter_df.to_json(orient='records')

# Helper Function
def convert_to_float(item):
    if item != item:  # Check for NaN
        return np.NaN
    
    item = item.strip('()')
    if 'B' in item:
        return float(item[:-1])
    elif 'M' in item:
        return float(item[:-1]) / 1000
    elif '%' in item:
        return float(item[:-1]) / 100
    else:
        return float(item)

if __name__ == '__main__':
    app.run(debug=True)
    

In [317]:
checker = ['Item',
 'Item',
 '30-Sep-2022',
 '31-Dec-2022',
 '31-Mar-2023',
 '30-Jun-2023',
 '30-Sep-2023',
 'Sales/Revenue']

checker[2:7]

['30-Sep-2022', '31-Dec-2022', '31-Mar-2023', '30-Jun-2023', '30-Sep-2023']

In [318]:
quarter_df = financial('tsla')
quarter_df

,Period Ending,Sales/Revenue,Sales Growth,Cost of Goods Sold (COGS) incl. D&A,COGS Growth,COGS excluding D&A,Depreciation & Amortization Expense,Depreciation,Amortization of Intangibles,Gross Income,...,Net Income Available to Common,EPS (Basic),EPS (Basic) Growth,Basic Shares Outstanding,EPS (Diluted),EPS (Diluted) Growth,Diluted Shares Outstanding,EBITDA,EBITDA Growth,EBITDA Margin
0,01-Jan-0001,25.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.44,...,NaN,2.49,NaN,3.18,NaN,NaN,3.49,NaN,NaN,NaN
1,31-Dec-2022,24.32,-0.0337,18.54,NaN,17.55,0.989,0.670,0.319,5.78,...,3.72,1.18,-0.5274,3.16,1.07,NaN,3.47,4.92,NaN,NaN
2,31-Mar-2023,23.33,-0.0407,18.82,0.0149,17.77,1.050,0.722,0.324,4.51,...,2.52,0.80,-0.3248,3.17,0.73,-0.3229,3.47,3.71,-0.2465,NaN
3,30-Jun-2023,24.93,0.0685,20.39,0.0837,19.24,1.150,0.816,0.338,4.53,...,2.70,0.85,0.0718,3.17,0.78,0.0704,3.48,3.55,-0.0423,NaN
4,30-Sep-2023,23.35,-0.0633,19.17,-0.0599,17.94,1.240,0.897,0.338,4.18,...,1.85,0.58,-0.3163,3.18,0.53,-0.3182,3.49,3.00,-0.1559,0.1284


In [319]:
quarter_df['Sales/Revenue'] / quarter_df["Depreciation"]

0          NaN
1    36.298507
2    32.313019
3    30.551471
4    26.031215
dtype: float64

In [320]:
for col in quarter_df.columns:
    print(col)

Period Ending
Sales/Revenue
Sales Growth
Cost of Goods Sold (COGS) incl. D&A
COGS Growth
COGS excluding D&A
Depreciation & Amortization Expense
Depreciation
Amortization of Intangibles
Gross Income
Gross Income Growth
Gross Profit Margin
SG&A Expense
SGA Growth
Research & Development
Other SG&A
Other Operating Expense
Unusual Expense
EBIT after Unusual Expense
Non Operating Income/Expense
Non-Operating Interest Income
Equity in Affiliates (Pretax)
Interest Expense
Interest Expense Growth
Gross Interest Expense
Interest Capitalized
Pretax Income
Pretax Income Growth
Pretax Margin
Income Tax
Income Tax - Current Domestic
Income Tax - Current Foreign
Income Tax - Deferred Domestic
Income Tax - Deferred Foreign
Income Tax Credits
Equity in Affiliates
Other After Tax Income (Expense)
Consolidated Net Income
Minority Interest Expense
Net Income
Net Income Growth
Net Margin Growth
Extraordinaries & Discontinued Operations
Extra Items & Gain/Loss Sale Of Assets
Cumulative Effect - Accounting C